In [ ]:
#1. What are the questions that members are asking about their membership benefits? 
#Create a list of the top 10 and calculate an estimate of % of customers asking this. 

In [56]:
import pandas as pd
from rapidfuzz import fuzz, process

convo = pd.read_csv("Sample 100 Convos - Clinikk - To Share - sample-redacted-convos.csv")

In [57]:
#keyword list.
membership_benefit_keywords = [
    # Membership Activation / Status
    "about my membership",
    "activate my membership",
    "activation help",
    "membership pending",
    "membership status",
    "start my membership",

    # Membership Cancellation
    "cancel my membership",
    "cancel my plan",
    "stop my membership",
    "close this plan",
    "not interested to continue",
    "membership cancel",
    "band karna h",
    "plan got cancelled",

    # Insurance / Policy Benefits
    "insurance details",
    "insurance document",
    "policy copy",
    "policy document",
    "included in insurance",
    "insurance coverage",
    "pre post medical bill",
    "included services",
    "benefits of membership",
    "services in the plan",
    "plan details",
    "health issue claim",
    "what is covered",
    "what all do i get",

    # Payments / Auto‑Debit
    "auto debit",
    "stop auto debit",
    "auto deduction",
    "deducted money",
    "money deducted",
    "refund money",
    "cancel auto debit",
    "cancel auto payment",
    "payment deducted",
    "every month payment",
    "mai har mahine pay nahi kar sakta",

    # Renewals
    "renew my policy",
    "renew my membership",
    "renewal link",
    "help with renewal",

    # Misc
    "subscription id",
    "nxt payment due date",
    "payment link not showing",
    "help with payment",

]

pattern = r"|".join([re.escape(kw) for kw in membership_benefit_keywords])

#Filter only user messages that match at least one keyword
user_msgs = convo[convo["sender_type"] == "user"].dropna(subset=["content"])

benefit_msgs = user_msgs[user_msgs["content"].str.contains(pattern, flags=re.IGNORECASE, na=False)]

canonical_questions = []

def fuzzy_bucket(msg, threshold=90):
    global canonical_questions
    best = process.extractOne(msg, canonical_questions, scorer=fuzz.token_sort_ratio)
    if best and best[1] >= threshold:
        return best[0]
    canonical_questions.append(msg)
    return msg

benefit_msgs["fuzzy_bucket"] = benefit_msgs["content"].apply(fuzzy_bucket)

#Quick sanity‑check
print("Matched rows:")
benefit_msgs[['content']]

Matched rows:


C:\Users\bhoom\AppData\Local\Temp\ipykernel_9012\4191127897.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  benefit_msgs["fuzzy_bucket"] = benefit_msgs["content"].apply(fuzzy_bucket)


,content
8,About my membership
12,Membership cancel ayindi
17,Cancel my plan
56,Stop auto debit
68,Help with payment
96,Cancel auto payment
120,Sir health insurance details
141,I am not interested to continue it due to mone...
149,Upcoming payment link not showing
150,Upcoming payment link not showing


In [58]:
#Aggregate identical *questions* (exact text) and count unique users
q_user_counts = benefit_msgs.groupby("fuzzy_bucket")['sender_id'].nunique().sort_values(ascending=False)
q_user_counts

fuzzy_bucket
Help with payment                                                                                               2
About my membership                                                                                             1
Cancel my plan                                                                                                  1
Cancel auto payment                                                                                             1
Hamara bike wala aur auto wala active hai auto walon Ko hatana hai band karna hai vah kaise hoga bataiye aap    1
Hi I need help with renewal of my plan                                                                          1
Hi I want to cancel my membership. Subscription id : 67ff74c9e164438daeaa3d19                                   1
Hi I want to cancel my membership. Subscription id : [redacted]                                                 1
I am not interested to continue it due to money problem.                   

In [59]:
total_unique_users = user_msgs["sender_id"].nunique()
benefit_customers = benefit_msgs["sender_id"].nunique()

percent_per_q = (q_user_counts / total_unique_users * 100).round(2)

# Prepare final summary DataFrame
top_q_df = pd.DataFrame({
    "question_text": q_user_counts.index,
    "unique_users": q_user_counts.values,
    "%_of_all_users": percent_per_q.values,
})

# Show top 10 grouped questions
top_q_df.head(10)


,question_text,unique_users,%_of_all_users
0,Help with payment,2,2.30
1,About my membership,1,1.15
2,Cancel my plan,1,1.15
3,Cancel auto payment,1,1.15
4,Hamara bike wala aur auto wala active hai auto...,1,1.15
5,Hi I need help with renewal of my plan,1,1.15
6,Hi I want to cancel my membership. Subscriptio...,1,1.15
7,Hi I want to cancel my membership. Subscriptio...,1,1.15
8,I am not interested to continue it due to mone...,1,1.15
9,I want delete auto pay facility and refund my ...,1,1.15


In [60]:
percent_asking_benefits = round((benefit_customers / total_unique_users) * 100, 2)
print("\n--- Summary ---")
print("Total Unique Customers:", total_unique_users)
print("Customers Asking About Benefits:", benefit_customers)
print("Percentage:", percent_asking_benefits, "%")


--- Summary ---
Total Unique Customers: 87
Customers Asking About Benefits: 19
Percentage: 21.84 %


In [61]:
#2. What are new services that customers are requesting?  
#Create a list of the top 10 and calculate an estimate of % of customers requesting this.

In [62]:
#keyword list.
services_requested_keywords = [
    "online consultation",
    "need doctor call",
    "schedule appointment",
    "scan request",
    "send me the scanner",
    "get prescription",
    "i havent received my prescription",
    "need prescription",
    "not able to find my prescription",
    "consultation request",
    "speak to doctor",
    "doctor hasnt called",
    "health care centre",
    "need bill",
    "send me bill",
    "payment receipt",
    "share insurance plan",
    "upload documents",
    "download policy",
    "get report",
    "need precautions",
    "send precautions",
    "claim insurance",
    "clem insurance",
    "balance my claim amount",
    "birth date wrong",
    "name correction",
    "connect me with pediatrician",
    'call'
]

pattern = r"|".join([re.escape(kw) for kw in services_requested_keywords])

#Filter only user messages that match at least one keyword
user_msgs = convo[convo["sender_type"] == "user"].dropna(subset=["content"])

new_service_msgs = user_msgs[user_msgs["content"].str.contains(pattern, flags=re.IGNORECASE, na=False)]

canonical_questions = []

def fuzzy_bucket(msg, threshold=90):
    global canonical_questions
    best = process.extractOne(msg, canonical_questions, scorer=fuzz.token_sort_ratio)
    if best and best[1] >= threshold:
        return best[0]
    canonical_questions.append(msg)
    return msg

# Apply fuzzy grouping
new_service_msgs["fuzzy_bucket"] = new_service_msgs["content"].apply(fuzzy_bucket)

#Quick sanity‑check
print("Matched rows:")
new_service_msgs[['content']]

Matched rows:


C:\Users\bhoom\AppData\Local\Temp\ipykernel_9012\2236759818.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_service_msgs["fuzzy_bucket"] = new_service_msgs["content"].apply(fuzzy_bucket)


,content
113,Hi sir please call me
118,Sir please call me
171,Aap thoda call kar leti to mere liye achcha hota
172,Call se baat Ho jaati
227,Hi I need help.plz call me
304,And that person who take payment on call she c...
312,She call me and pay it's a one time payment
334,That person calls me
399,Morning please call me I will explain
428,Please call me


In [63]:
#Aggregate identical *questions* (exact text) and count unique users
q_user_counts = new_service_msgs.groupby("fuzzy_bucket")["sender_id"].nunique().sort_values(ascending=False)
q_user_counts

fuzzy_bucket
Aap thoda call kar leti to mere liye achcha hota                          1
And that person who take payment on call she commit                       1
Call se baat Ho jaati                                                     1
Can u please inform the doctor to call me to this number [redacted]       1
Doctor have not called me yet                                             1
Hello!\nI need an online consultation                                     1
Hi I need help.plz call me                                                1
Hi sir please call me                                                     1
Hi will the doctor only call me?                                          1
How much balance my claim amount                                          1
I am not able to find my prescription                                     1
I'm in a very bad pain and none of your doctor had called me              1
Morning please call me I will explain                                     1

In [64]:
total_unique_users = user_msgs["sender_id"].nunique()
service_customers = new_service_msgs["sender_id"].nunique()
percent_per_q = (q_user_counts / total_unique_users * 100).round(2)

# Final DataFrame
top_q_df = pd.DataFrame({
    "question_text": q_user_counts.index,
    "unique_users": q_user_counts.values,
    "%_of_all_users": percent_per_q.values,
})

# Output top queries
top_q_df.head(10)

,question_text,unique_users,%_of_all_users
0,Aap thoda call kar leti to mere liye achcha hota,1,1.15
1,And that person who take payment on call she c...,1,1.15
2,Call se baat Ho jaati,1,1.15
3,Can u please inform the doctor to call me to t...,1,1.15
4,Doctor have not called me yet,1,1.15
5,Hello!\nI need an online consultation,1,1.15
6,Hi I need help.plz call me,1,1.15
7,Hi sir please call me,1,1.15
8,Hi will the doctor only call me?,1,1.15
9,How much balance my claim amount,1,1.15


In [65]:
percent_asking_services = round((service_customers / total_unique_users) * 100, 2)
print("\n--- Summary ---")
print("Total Unique Customers:", total_unique_users)
print("Customers Asking About New Services:", service_customers)
print("Percentage:", percent_asking_services, "%")


--- Summary ---
Total Unique Customers: 87
Customers Asking About New Services: 13
Percentage: 14.94 %


In [14]:
#3. Filter messages where the customers ask policy-questions and the agents’ responses. 
# Your filtered dataset should have the question(s) by the user and the answer(s) by the agent. 

In [66]:
#keyword list.
policy_keywords = [
    "insurance policy",
    "policy copy",
    "what is covered",
    "included in insurance",
    "claim amount",
    "coverage details",
    "reimbursement",
    "health issue claim",
    "can I claim",
    "policy status",
    "renewal of policy",
    "policy cancel",
    "insurance refund",
    "refund after cancellation",
    "deducted money refund",
    "policy documents",
    "insurance claim help",
]

pattern = r"|".join([re.escape(kw) for kw in services_requested_keywords])
#pattern = "(" + "|".join(re.escape(kw) for kw in services_requested_keywords) + ")"

#Filter user messages that match at least one keyword
user_msgs = convo[convo["sender_type"] == "user"].dropna(subset=["content"])
#Filter agent messages 
agent_msgs = convo[convo["sender_type"] == "agent"].dropna(subset=["content"])


policy_qs = user_msgs[user_msgs["content"].str.contains(pattern, flags=re.IGNORECASE, na=False)] #user

policy_qas = [] #agent

for idx, row in policy_qs.iterrows():
    convo_id = row["conversation_id"]
    timestamp = row["message_sent_at"]
    
    # Find agent reply after this message in the same conversation
    agent_replies = agent_msgs[
        (agent_msgs["conversation_id"] == convo_id) &
        (agent_msgs["message_sent_at"] > timestamp)
    ].sort_values("message_sent_at")
    
    if not agent_replies.empty:
        first_reply = agent_replies.iloc[0]
        policy_qas.append({
            "conversation_id": convo_id,
            "question": row["content"],
            "answer": first_reply["content"],
            "user_time": row["message_sent_at"],
            "agent_time": first_reply["message_sent_at"],
        })

#Create DataFrame
policy_qas_df = pd.DataFrame(policy_qas)
print(f"Total policy-related Q&A pairs found: {len(policy_qas_df)}")
policy_qas_df

Total policy-related Q&A pairs found: 21


,conversation_id,question,answer,user_time,agent_time
0,75174,Hi sir please call me,Hello! I’m Aswini from the Care Team at Clinik...,"April 21, 2025, 9:02 AM","April 21, 2025, 9:04 AM"
1,75174,Sir please call me,"By selecting the self-plan, you can enjoy cove...","April 21, 2025, 9:06 AM","April 21, 2025, 9:08 AM"
2,75168,Aap thoda call kar leti to mere liye achcha hota,Sorry sir isme call option nahiye chat karna p...,"April 21, 2025, 8:29 AM","April 21, 2025, 8:30 AM"
3,75168,Call se baat Ho jaati,Sorry sir isme call option nahiye chat karna p...,"April 21, 2025, 8:29 AM","April 21, 2025, 8:30 AM"
4,75163,Hi I need help.plz call me,Hello! I’m Aswini from the Care Team at Clini...,"April 21, 2025, 7:49 AM","April 21, 2025, 8:02 AM"
5,75155,And that person who take payment on call she c...,You bought this plan through [redacted] for ju...,"April 21, 2025, 8:08 AM","April 21, 2025, 8:10 AM"
6,75155,She call me and pay it's a one time payment,"With this health insurance, you'll need to mak...","April 21, 2025, 8:10 AM","April 21, 2025, 8:11 AM"
7,75155,That person calls me,Of course! Is there anything else that I can h...,"April 21, 2025, 8:20 AM","April 21, 2025, 8:22 AM"
8,75143,Morning please call me I will explain,"If you explain your issue in the chat, we will...","April 20, 2025, 11:09 PM","April 20, 2025, 11:10 PM"
9,75141,Please call me,Kya hum jan sakthey hain keoun apko a policy b...,"April 20, 2025, 10:51 PM","April 20, 2025, 10:53 PM"


In [ ]:
#4. Filter messages where the customers ask questions about service availability in our clinics and the agents’ responses. 
# Your filtered dataset should have the question(s) by the user and the answer(s) by the agent.

In [67]:
#keyword list.
services_available_keywords = [
    "clinic open",
    "is today clinic open",
    "nearby clinic",
    "health care centre",
    "available at clinic",
    "can i visit clinikk",
    "clinikk banashankari",
    "doctor available",
    "hospital available",
    "limited hospital available",
    "scan centre available",
    "test available",
    "location available",
    "clinic timing",
]

pattern = r"|".join([re.escape(kw) for kw in services_available_keywords])
#pattern = "(" + "|".join(re.escape(kw) for kw in services_available_keywords) + ")"

#Filter user messages that match at least one keyword
user_msgs = convo[convo["sender_type"] == "user"].dropna(subset=["content"])
#Filter agent messages 
agent_msgs = convo[convo["sender_type"] == "agent"].dropna(subset=["content"])


services_available_qs = user_msgs[user_msgs["content"].str.contains(pattern, flags=re.IGNORECASE, na=False)] #user

services_available_qas = [] #agent

for idx, row in services_available_qs.iterrows():
    convo_id = row["conversation_id"]
    timestamp = row["message_sent_at"]
    
    # Find agent reply after this message in the same conversation
    agent_replies = agent_msgs[
        (agent_msgs["conversation_id"] == convo_id) &
        (agent_msgs["message_sent_at"] > timestamp)
    ].sort_values("message_sent_at")
    
    if not agent_replies.empty:
        first_reply = agent_replies.iloc[0]
        services_available_qas.append({
            "conversation_id": convo_id,
            "question": row["content"],
            "answer": first_reply["content"],
            "user_time": row["message_sent_at"],
            "agent_time": first_reply["message_sent_at"],
        })

#Create DataFrame
services_available_qas_df = pd.DataFrame(services_available_qas)
print(f"Total services available-related Q&A pairs found: {len(services_available_qas_df)}")
services_available_qas_df

Total services available-related Q&A pairs found: 3


,conversation_id,question,answer,user_time,agent_time
0,75105,Is today clinic open?,Hello! I’m Fardeen from the Care Team at Clini...,"April 20, 2025, 4:50 PM","April 20, 2025, 4:51 PM"
1,74940,Good Evening \nCan I get an appointment schedu...,Hello! I’m Ashwini from the Care Team at Clini...,"April 19, 2025, 4:35 PM","April 19, 2025, 4:36 PM"
2,74940,Good Evening \nCan I get an appointment schedu...,Could you please let me know the name of the p...,"April 19, 2025, 4:37 PM","April 19, 2025, 4:39 PM"
